# Retrieving Simulator Logs from Log Analytics for Analysis

This notebook shows you how to access the logs from your simulators, and provides a few sample analyses. Episode level and iteration level logs during training and assessment are stored on [Log Anlytics](https://docs.microsoft.com/en-us/azure/azure-monitor/platform/data-platform-logs). The logs on Azure Monitor can be accessed using the query language [KQL](https://docs.microsoft.com/en-us/sharepoint/dev/general-development/keyword-query-language-kql-syntax-reference). 

The notebook uses the jupyter extension [`Kqlmagic`](https://github.com/Microsoft/jupyter-Kqlmagic), to automatically query the data directly within the notebook. More info + more complete functionality on the ` Kqlmagic` extension is available on its official [docs](https://docs.microsoft.com/en-us/azure/data-explorer/kqlmagic).

## Description

Simulator logs are enabled by default for brains trained during assessment. For training they must be enabled via the CLI command [bonsai brain version start-logging](https://docs.microsoft.com/en-us/bonsai/cli/brain/version/start-logging).

## Prerequisites

1. Install requirements: an [enviroment.yml](https://gist.github.com/akzaidi/ed687b492b0f9e77682b0a0a83397659/) is provided with a complete description of the packages needed to run this notebook using Anaconda. Please install Anaconda (or miniconda) and run `conda env update -f environment.yml` in the folder where you download the linked `environment.yml` file.
2. If your azure subscription has not yet been registered to allow Log Analytics workspace resource provider, it needs to be registered manually using the azure-cli:
    1. Determine if registering is required. <SUBCRIPTION_ID> can be found on preview.bons.ai by clicking on id/Workspace info. 
    ```
    az provider show --namespace 'Microsoft.OperationalInsights' -o table --subscription <SUBCRIPTION_ID>
    ```
    2.  If the registrationState is `Registered`, you can skip this step. If not registered, we will need to register it. This is a one-time step per subscription and user will need owner-level permission. If you don't have the appropriate permission, work with your IT admin to execute that step.

    ```
    az login
    az provider register --namespace 'Microsoft.OperationalInsights' --subscription <SUBCRIPTION_ID>
    ```
### Note

A Log Analytics workspace will get created on Azure if it does not already exist in the provisioned Bonsai workspace.
 
## Usage

1. Start a simulator instance for Brain training: 
    1. For managed simulators
        1. Add the simulator instance to your inkling using the `package` statement, or start training and select the simulator instance using the web
        2. _or_ use the CLI: [`bonsai brain version start-training --simulator-package-name`](https://docs.microsoft.com/en-us/bonsai/cli/brain/version/start-training)
    2. For unmanaged simulators:
        1. register a sim by launching your sim. For example `python main.py`
        2. start brain training `bonsai brain version start-training --name <BRAIN_NAME>`
        3. connect your registered sim to a brain `bonsai simulator unmanaged connect --simulator-name <SIM_NAME> --brain-name <BRAIN_NAME> --version <VERSION_#> --action Train --concept-name <CONCEPT_NAME>`
        4. Find the `session-id` of un-managed sim using Bonsai CLI: `bonsai simulator unmanaged list`
2. Enable logging using the CLI:
    1. For managed simulators, add the `-m` parameter for managed simulator logging and the number of instances to log:
        `bonsai brain version start-logging -m -s <NUM_INSTANCES> -n <BRAIN_NAME>`
        You can also optionally request for system logs with the `-l` parameter.
    2. For unmanaged simulators, you should specify the `session-id`: 
        `bonsai brain version start-logging -n <BRAIN_NAME> --session-id <SESSION_ID>`
3. Logs will start populating the Log Analytics workspace 3-5 minutes after starting logging
4. Obtain your KQL workspace-id by navigating to the KQL resource in the Azure Portal:
    ![log-analytics-azure-portal](log-analytics-id.png)

## Getting Started

### Load Kqlmagic Extension

The following cell loads the `Kqlmagic` extension into your notebook and sets the maximum number of rows to display in the output to five (the result will not be truncated however, other than by KQL's [query limit](https://docs.microsoft.com/en-us/azure/data-explorer/kusto/concepts/querylimits)). Whenever you want to use `KQL` in a cell, prepend that cell with the statem `%%kql`.

In [ ]:
%reload_ext Kqlmagic
%config Kqlmagic.display_limit = 5 #limiting the number of rows displayed (full rows will still be stored)

### Login to the log analytics workspace

Retrieve the `LOG_ANALYTICS_WORKSPACE_ID` using the instructions provided above. Please **note**, the log analytics workspace-id is the `workspace-id` of the log analytics workspace, **not** your Bonsai workspace!


In [ ]:
LOG_ANALYTICS_WORKSPACE_ID = "ea345eb0-8961-48c2-8656-be7a73f0df32"
ALIAS = 'jomcdowe' # could be anything, but should not be empty
%kql loganalytics://code;workspace=LOG_ANALYTICS_WORKSPACE_ID;alias=ALIAS

In [ ]:
# Enes "9e69f3b2-b326-4165-9dae-95bbb3e19244" 
# Hossein "149ae47e-aac6-495a-812c-2c3dd3018f20"
# Journey "ea345eb0-8961-48c2-8656-be7a73f0df32"

## Iteration and Episode Level Logs

There are four tables generated by the Bonsai service and sent to your Log Analytics workspace. These are:

1. `EpisodeLog_CL` which contains your episode-level logs, such as information about the starting simulator configuration of your episodes, your aggregate goal metrics, etc.
2. `IterationLog_CL` which contains the interation-level logs of your simulator, including the actions received from the platform, and the states emitted from your simulators.
3. `ContainerEvent_CL` (**only for managed simulators that were logged with system logs**): container event logs such as image pull time, build time, etc.
4. `ContainerInstanceLog_CL` (**only for managed simulators that were logged with system logs**): system logs from your simulators.

Let's extract both iteration (`IterationLog_CL`) and episode-level (`EpisodeLog_CL`) logs and join them together via a KQL query. We then export the query results to a pandas dataframe which we can then manipulate and analyze.

**Note**: if this is the first time you're using logging you may need to wait for the first episode to finish so that episode-level logs are created and filled with at least 1 row of data.

### Specify Query Variables and Query

In order to make our query specific to a brain version and brain name, we can create variables containing those values. The following cell does exactly that, and in addition specific a max number of rows to query.

<div class="alert alert-block alert-danger">
<b>WARNING: KQL queries are case-sensitive and the data stored in your analytics workspace is saved as lowercase strings. Brain name and assessment name must be set as lowercase or the query options noted below will fail to find any data.
</div>

In [ ]:
number_of_rows = 100000
brain_name = "scg-fun".lower()
brain_version = 71

#### Query

Here is our `KQL` query. Observe that we first filter the `EpisodeLog_CL` table to the brain name and brain version we defined above, and then join it with the `IterationLog_CL` table using the unique `EpisodeId_g` index. Finally, we select (using `project`) a set of columns we want to retain in our resulting dataframe.

In [ ]:
%%kql
let _number_of_rows = number_of_rows;
let _brain_name = brain_name;
let _brain_version = brain_version;
EpisodeLog_CL
  | where BrainName_s == _brain_name and BrainVersion_d == _brain_version
  | where  TimeGenerated > ago(60d)
  | join kind=inner (
      IterationLog_CL
      | sort by Timestamp_t desc
      ) on EpisodeId_g
  | take _number_of_rows
  | project 
      Timestamp = Timestamp_t,
      SessionIndex = SessionId_s,
      EpisodeIndex = EpisodeIndex_d,
      IterationIndex = IterationIndex_d,
      EpisodeType_s = EpisodeType_s,
      BrainName = BrainName_s,
      BrainVersion = BrainVersion_d,
      SimState = parse_json(SimState_s),
      SimAction = parse_json(SimAction_s),
      SimConfig = parse_json(SimConfig_s),
      Reward = Reward_d,
      CumulativeReward = CumulativeReward_d,
      Terminal = Terminal_b,
      LessonIndex = LessonIndex_d,
      GoalMetrics = parse_json(GoalMetrics_s),
      EpisodeType = EpisodeType_s,
      FinishReason = FinishReason_s
  | order by SessionIndex, EpisodeIndex asc, IterationIndex asc

### Magic

Magically, the output of the result is saved to a new python variable called `_kql_raw_result`. We can convert this to a pandas dataframe:

In [ ]:
# convert query results in a dataframe
iter_df = _kql_raw_result_.to_dataframe()
iter_df.head(5)

In [ ]:
iter_df['Timestamp'].head()

In [ ]:
iter_df['Timestamp'].tail()

In [ ]:
iter_df.describe()

In [ ]:
iter_df[iter_df['CumulativeReward'] >= 50].describe()

In [ ]:
len(iter_df['EpisodeIndex'].unique())

In [ ]:
len(iter_df['SessionIndex'].unique())

In [ ]:
iter_df['EpisodeIndex'].max()

In [ ]:
num_eps = 0
for sessidx in iter_df['SessionIndex'].unique():
    temp_df = iter_df[iter_df['SessionIndex']==sessidx]
    num_ep = temp_df['EpisodeIndex'].max()
    num_eps += num_ep
    
print(num_eps)

## Converting Nested Array Into New Columns

Notice that the array-data as stored in `SimState`, `SimAction` and `SimConfig` are dictionaries. You can cast them into new columns using the operations below:

In [ ]:
import pandas as pd
def format_kql_logs(df: pd.DataFrame) -> pd.DataFrame:
    ''' Function to format a dataframe obtained from KQL query.
        Output format: keeps only selected columns, and flatten nested columns [SimAction, SimState, SimConfig]

        Parameters
        ----------
        df : DataFrame
            dataframe obtained from running KQL query then exporting `_kql_raw_result_.to_dataframe()`
    '''
    selected_columns = ["Timestamp","EpisodeIndex", "IterationIndex", "Reward", "Terminal", "SimState", "SimAction", "SimConfig", "SessionIndex", "CumulativeReward"]
    nested_columns =  [ "SimState", "SimAction", "SimConfig"]
    df_selected_columns = df[selected_columns]
    series_lst = []
    ordered_columns = ["Timestamp","EpisodeIndex", "IterationIndex", "Reward", "Terminal", "IterationSpeed_s", "SessionIndex", "CumulativeReward"]
    for i in nested_columns:
        new_series = df_selected_columns[i].apply(pd.Series)
        column_names = new_series.columns.values.tolist()
        series_lst.append(new_series)
        if len(column_names) > 0:
            ordered_columns.extend(column_names)
        del(df_selected_columns[i])

    series_lst.append(df_selected_columns)
    formated_df = pd.concat(series_lst, axis=1)
    formated_df = formated_df.sort_values(by='Timestamp',ascending=True) # reorder df based on Timestamp
    formated_df.index = range(len(formated_df)) # re-index
    formated_df['Timestamp']=pd.to_datetime(formated_df['Timestamp']) # convert Timestamp to datetime
    formated_df['IterationSpeed_s']=formated_df['Timestamp'].diff().dt.total_seconds() # convert Timestamp to datetime

    formated_df = formated_df[ordered_columns]
    
    return formated_df.sort_values(by=["EpisodeIndex", "IterationIndex"])


In [ ]:
df = format_kql_logs(iter_df)

In [ ]:
df[df['SessionIndex'] == '1575438888_10.244.93.231'][['BR_201','BR_221', 'C2', 'IterationIndex']].describe()

In [ ]:
num_eps = 0
for sessidx in df['SessionIndex'].unique():
    temp_df = df[df['SessionIndex']==sessidx]
    num_ep = temp_df['EpisodeIndex'].max()
    num_eps += num_ep
    
print(num_eps)

In [ ]:
df.describe()

In [ ]:
len(df['SessionIndex'].unique())

In [ ]:
df.sort_values(by=["EpisodeIndex", "IterationIndex"])

In [ ]:
# per step reward
df.hist(column='reward_value', bins=10)

In [ ]:
df_last = pd.DataFrame({})
for sess in iter_df['SessionIndex'].unique():
    df_sess = iter_df[iter_df['SessionIndex']==sess]
    for ep in range(1, int(iter_df['EpisodeIndex'].max()+1)):
        last_iter = df_sess[ (df_sess['EpisodeIndex']==ep) & (df_sess['IterationIndex'] == len(df_sess[df_sess['EpisodeIndex']==ep]))]
        df_last = pd.concat([df_last, last_iter], sort=False)

In [ ]:
df_last = format_kql_logs(df_last)

In [ ]:
df_of_interest = df_last[df_last['CumulativeReward'] >= 56]
df_of_interest.describe()

In [ ]:
num_eps = 0
for sessidx in df_of_interest['SessionIndex'].unique():
    temp_df = df_of_interest[df_last['SessionIndex']==sessidx]
    num_ep = len(temp_df['EpisodeIndex'].unique())
    num_eps += num_ep
    
print(num_eps)

In [ ]:
# 56
537 / 14698

In [ ]:
# 50
7365 / 14698

In [ ]:
# episode return reward
df_of_interest.hist(column='CumulativeReward', bins=10)

# Ranges through # of samples

In [ ]:
df_review = pd.DataFrame({})

cols = [
    'SessionIndex',
    'EpisodeIndex',
    'IterationIndex',
    'Terminal',
    'deltaMI_201',
    'deltaMI_221',
    'current_H2_201',
    'current_H2_221',
    'deltaPressure',
    'MI_201',
    'MI_221',
    'C2',
    'BR_201',
    'BR_221',
    #'Density_221',
    'MI_201_target_range_low',
    'MI_201_target_range_high',
    'MI_221_target_range_low',
    'MI_221_target_range_high',
    'step_count',
    'pressure_error',
    'sim_error',
    'sim_action_H2_201',
    'sim_action_H2_221',
    'sim_action_C2',
    'sim_action_C4_241',
    'sim_action_C4_221',
    'sim_action_BR_201',
    'sim_action_BR_241',
    'sim_action_BR_221',
    'H2_201',
    'H2_221',
    'sigma',
    'training_grade',
    'Reward'
]

state_cols = [
    'deltaMI_201',
    'deltaMI_221',
    'current_H2_201',
    'current_H2_221',
    'deltaPressure',
    'MI_201',
    'MI_221',
    'C2',
    'BR_201',
    'BR_221',
    #'Density_221',
    'MI_201_target_range_low',
    'MI_201_target_range_high',
    'MI_221_target_range_low',
    'MI_221_target_range_high',
    'step_count',
    'pressure_error',
    'sim_error',
    'sim_action_H2_201',
    'sim_action_H2_221',
    'sim_action_C2',
    'sim_action_C4_241',
    'sim_action_C4_221',
    'sim_action_BR_201',
    'sim_action_BR_241',
    'sim_action_BR_221',
]

action_cols = [
    'H2_201',
    'H2_221',
]

config_cols = [
    'sigma',
    'training_grade'
]

for sess in iter_df['SessionIndex'].unique():
#for sess in ['149204247_10.244.8.228']:
    print("doing sess {}".format(sess))
    df_session = iter_df[iter_df['SessionIndex'] == sess]
    df_session = format_kql_logs(df_session)
    df_session.sort_values(by=["EpisodeIndex", "IterationIndex"])
    for ep in range(1, int(max(df_session['EpisodeIndex']))+1):
    #for ep in range(5, 6):
        print("doing ep {}".format(ep))
        df_flat = df_session[df_session['EpisodeIndex'] == ep]
        if len(df_flat) > 1:
            if df_flat['IterationIndex'].iloc[1] == 1:
                print('Truncating ep because duplicate')
            elif df_flat['IterationIndex'].iloc[0] == 1:
                df_flat = df_flat[cols]
                df_review = pd.concat([df_review, df_flat])
                df_review['pressure_error'] = df_review['pressure_error'].map(int)
                df_review['sim_error'] = df_review['sim_error'].map(int)
            else:
                print('Truncating ep because did not start at IterationIndex=1')
        else:
            print('Empty episode')

In [ ]:
for col in state_cols:
    print(df_review[col].describe())

In [ ]:
#ranges_to_eval = [5000, 20000, 40000, 80000]
#ranges_to_eval = [5000, 10000, 20000, 36228]
ranges_to_eval = [-1]
df_dict = {}
for i in range(len(ranges_to_eval)):
    rng_to_slice = int(1.2 * ranges_to_eval[i])
    df_dict[str(ranges_to_eval[i])] = df_review[:rng_to_slice]

In [ ]:
def plot_ranges(ranges_to_eval, state_cols, dfs):
    import matplotlib.pyplot as plt
    col_maxes = pd.DataFrame({})
    col_mins = pd.DataFrame({})
    for key, df in dfs.items():
        temp_max_df = {}
        temp_min_df = {}
        for col in state_cols:
            temp_max_df[col] = [df[col].describe()["max"]]
            temp_min_df[col] = [df[col].describe()["min"]]
        col_maxes = pd.concat([col_maxes, pd.DataFrame(temp_max_df)])
        col_mins = pd.concat([col_mins, pd.DataFrame(temp_min_df)])
    
    for col in state_cols:
        fig, ax = plt.subplots(1, 1)
        ax.plot(ranges_to_eval, col_maxes[col], marker='*', label=str(col)+'_max')
        ax.plot(ranges_to_eval, col_mins[col], marker='*', label=str(col)+'_min')
        
        ax.legend()
        plt.show()

In [ ]:
plot_ranges(ranges_to_eval, state_cols, df_dict)

# Convert subset to NeuralSim Dev tool

In [ ]:
num_samples = -1
df_to_convert = df_dict[str(num_samples)]

In [ ]:
import os
import numpy as np
import random

isdir = os.path.isdir("training")
if isdir:
    pass
else:
    os.makedirs("training")
    
isdir = os.path.isdir("validation")
if isdir:
    pass
else:
    os.makedirs("validation")

In [ ]:
for sess in df_to_convert['SessionIndex'].unique():
    print("doing sess {}".format(sess))
    df_session = df_to_convert[df_to_convert['SessionIndex'] == sess]
    df_session.sort_values(by=["EpisodeIndex", "IterationIndex"])
    for ep in df_session['EpisodeIndex'].unique():
        print("doing ep {}".format(ep))
        df_flat = df_session[df_session['EpisodeIndex'] == ep]
        df_flat = df_flat[cols]
        df_ep = pd.DataFrame({})
        df_ep['EpisodeIndex'] = '"' + sess + '_ep_' +df_flat['EpisodeIndex'].map(str) + '"'
        df_ep['IterationIndex'] = df_flat['IterationIndex'].astype(int)
        df_ep['is_initial_state'] = df_flat.eval("IterationIndex == 1")
        df_ep['is_terminal_state'] = df_flat['Terminal']
        df_ep['is_initial_state'] = df_ep['is_initial_state'].replace(True, 'true')
        df_ep['is_initial_state'] = df_ep['is_initial_state'].replace(False, 'false')
        df_ep['is_terminal_state'] = df_ep['is_terminal_state'].replace(True, 'true')
        df_ep['is_terminal_state'] = df_ep['is_terminal_state'].replace(False, 'false')
        df_ep['state'] = df_flat[state_cols].to_numpy().tolist()
        df_ep['action'] = df_flat[action_cols].to_numpy().tolist()
        df_ep['action'].iloc[0] = []
        df_ep['config'] = df_flat[config_cols].to_numpy().tolist()
        # since empty config, need 0
        #df_ep['config'] = df_ep['config'].apply(lambda x: x if x else 0)
        if np.random.uniform(0, 1) <= 0.1:
            df_ep.to_csv('./validation/{}_ep_{}.csv'.format(sess, int(ep)), index=False, header=False)
        else:
            df_ep.to_csv('./training/{}_ep_{}.csv'.format(sess, int(ep)), index=False, header=False)

In [ ]:
import glob, os
import shutil

training_data_count = 0
parent_dir = './training'
for file in glob.glob(os.path.join(parent_dir, '*.csv')):
    with open(file) as fname:
        df = pd.read_csv(fname)
    training_data_count += len(df)

parent_dir = './validation'

broken = False
while training_data_count < num_samples:
    for file in glob.glob(os.path.join(parent_dir, '*.csv')):
        with open(file) as fname:
            df = pd.read_csv(fname)
        data_count = len(df)
        if training_data_count + data_count < num_samples:
            training_data_count += data_count
            shutil.move(file, './training')
        else:
            training_data_count += data_count
            shutil.move(file, './training')
            print('Increased training dataset size from validation set to {}'.format(training_data_count))
            broken = True
            break
    if broken:
        break
    if glob.glob(os.path.join(parent_dir, '*.csv')):
        pass
    else:
        print('Out of files')
        print('Training dataset size now: {}'.format(training_data_count))
        break

In [ ]:
n_training_data = [5017, 10000, 20017, 30002]
champion_val_data = [48.22986602783203, 17.79674784342448, 13.800888061523438, 10.681306665593928]

import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1)

ax.plot(n_training_data, champion_val_data, label='champion val loss', marker='*')
ax.legend()
plt.show()

## Retrieving Logs for Assessment

Here is an example of extracting logs that were generated during assessment. We modify our above query by including the `AssessmentId_g` column, and filter only for those that were not empty, which should be when the simulators were being used for assessment:

In [ ]:
%%kql
let _number_of_rows = 1000;
let _brain_name = brain_name;
let _brain_version = brain_version;
EpisodeLog_CL
  | where BrainName_s == _brain_name and BrainVersion_d == _brain_version
  | where  TimeGenerated > ago(30d)
  | where AssessmentId_g != ""
  | join kind=inner (
      IterationLog_CL
      | sort by Timestamp_t desc
      ) on EpisodeId_g
//   | take _number_of_rows
  | project 
      Timestamp = Timestamp_t,
      AssessmentName = AssessmentName_s,
      SessionIndex = SessionId_s,
      EpisodeIndex = EpisodeIndex_d,
      IterationIndex = IterationIndex_d,
      EpisodeType_s = EpisodeType_s,
      BrainName = BrainName_s,
      BrainVersion = BrainVersion_d,
      SimState = parse_json(SimState_s),
      SimAction = parse_json(SimAction_s),
      SimConfig = parse_json(SimConfig_s),
      Reward = Reward_d,
      CumulativeReward = CumulativeReward_d,
      Terminal = Terminal_b,
      LessonIndex = LessonIndex_d,
      GoalMetrics = parse_json(GoalMetrics_s),
      EpisodeType = EpisodeType_s,
      FinishReason = FinishReason_s
  | order by AssessmentName, SessionIndex, EpisodeIndex asc, IterationIndex asc

In [ ]:
assess_df = _kql_raw_result_.to_dataframe()

In [ ]:
assess_df =  assess_df[assess_df['AssessmentName'] == 'custom_assessment_1']

In [ ]:
assess_df['SessionIndex'].unique()

In [ ]:
def plot_assess(df, SessionIndex, ep):
    df = df[df['SessionIndex'] == SessionIndex]
    df = df[df['EpisodeIndex'] == ep]
    df_flattened = format_kql_logs(df)
    
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(2, 1, figsize=(10,8), sharex=True)
    ax[0].plot(df_flattened['IterationIndex'].values, df_flattened['MI_201'].values, marker='*')
    ax[0].plot(df_flattened['IterationIndex'].values, df_flattened['MI_201_target_range_low'].values, 'r--')
    ax[0].plot(df_flattened['IterationIndex'].values, df_flattened['MI_201_target_range_high'].values, 'r--')
    ax[0].legend(['MI_201'])
    ax[0].set_ylabel('g/10min')
    
    ax[1].plot(df_flattened['IterationIndex'].values, df_flattened['MI_221'].values, marker='*')
    ax[1].plot(df_flattened['IterationIndex'].values, df_flattened['MI_221_target_range_low'].values, 'r--')
    ax[1].plot(df_flattened['IterationIndex'].values, df_flattened['MI_221_target_range_high'].values, 'r--')
    ax[1].legend(['MI_221'])
    ax[1].set_ylabel('g/10min')
    ax[1].set_xlabel('Iteration')

    plt.show()

In [ ]:
sessionindex = '616542713_10.244.36.191'
num_eps = len(assess_df[assess_df['SessionIndex'] == sessionindex]['EpisodeIndex'].unique())
for i in range(num_eps):
    plot_assess(assess_df, sessionindex, i+1)

## From ADE csv

In [ ]:
assess_df = pd.read_csv('sac_bline.csv')

In [ ]:
assess_df =  assess_df[assess_df['AssessmentName'] == 'custom_assessment_1']

In [ ]:
assess_df['SessionIndex'].unique()

In [ ]:
import pandas as pd
import ast
def format_csv_logs(df: pd.DataFrame) -> pd.DataFrame:
    ''' Function to format a dataframe obtained from KQL query.
        Output format: keeps only selected columns, and flatten nested columns [SimAction, SimState, SimConfig]

        Parameters
        ----------
        df : DataFrame
            dataframe obtained from running KQL query then exporting `_kql_raw_result_.to_dataframe()`
    '''
    selected_columns = ["Timestamp","EpisodeIndex", "IterationIndex", "Reward", "Terminal", "SimState", "SimAction", "SimConfig", "SessionIndex"]
    nested_columns =  [ "SimState", "SimAction", "SimConfig"]
    df_selected_columns = df[selected_columns]
    series_lst = []
    ordered_columns = ["Timestamp","EpisodeIndex", "IterationIndex", "Reward", "Terminal", "IterationSpeed_s", "SessionIndex"]
    for i in nested_columns:
        try:
            new_series = df_selected_columns[i].apply(ast.literal_eval).apply(pd.Series)
        except:
            df_selected_columns[i].fillna(value=str({key: str(np.nan) for key, value in ast.literal_eval(df_selected_columns[i][1]).items()}), inplace=True)
            new_series = df_selected_columns[i].apply(ast.literal_eval).apply(pd.Series)
        column_names = new_series.columns.values.tolist()
        series_lst.append(new_series)
        if len(column_names) > 0:
            ordered_columns.extend(column_names)
        del(df_selected_columns[i])

    series_lst.append(df_selected_columns)
    formated_df = pd.concat(series_lst, axis=1)
    formated_df = formated_df.sort_values(by='Timestamp',ascending=True) # reorder df based on Timestamp
    formated_df.index = range(len(formated_df)) # re-index
    formated_df['Timestamp']=pd.to_datetime(formated_df['Timestamp']) # convert Timestamp to datetime
    formated_df['IterationSpeed_s']=formated_df['Timestamp'].diff().dt.total_seconds() # convert Timestamp to datetime

    formated_df = formated_df[ordered_columns]
    
    return formated_df.sort_values(by=["EpisodeIndex", "IterationIndex"])

In [ ]:
df_flattened = format_csv_logs(assess_df)

In [ ]:
df_flattened.head()

In [ ]:
def plot_assess(df, SessionIndex, ep):
    df = df[df['SessionIndex'] == SessionIndex]
    df = df[df['EpisodeIndex'] == ep]
    df_flattened = format_kql_logs(df)
    
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(2, 1, figsize=(10,8), sharex=True)
    ax[0].plot(df_flattened['IterationIndex'].values, df_flattened['MI_201'].values, marker='*')
    ax[0].plot(df_flattened['IterationIndex'].values, df_flattened['MI_201_target_range_low'].values, 'r--')
    ax[0].plot(df_flattened['IterationIndex'].values, df_flattened['MI_201_target_range_high'].values, 'r--')
    ax[0].legend(['MI_201'])
    ax[0].set_ylabel('g/10min')
    
    ax[1].plot(df_flattened['IterationIndex'].values, df_flattened['MI_221'].values, marker='*')
    ax[1].plot(df_flattened['IterationIndex'].values, df_flattened['MI_221_target_range_low'].values, 'r--')
    ax[1].plot(df_flattened['IterationIndex'].values, df_flattened['MI_221_target_range_high'].values, 'r--')
    ax[1].legend(['MI_221'])
    ax[1].set_ylabel('g/10min')
    ax[1].set_xlabel('Iteration')

    plt.show()

In [ ]:
sessionindex = '616528460_10.244.36.191'
num_eps = len(assess_df[assess_df['SessionIndex'] == sessionindex]['EpisodeIndex'].unique())
for i in range(num_eps):
    plot_assess(assess_df, sessionindex, i+1)

# From Data

In [ ]:
def read_concat(path):
    import os
    df = pd.DataFrame({})
    for p in path:
        directories = os.listdir(p)
        for file in directories:
            new_df = pd.read_csv(p+'/'+file)
            df = pd.concat([df, new_df])
    
    return df

In [ ]:
df = read_concat(['./training', './validation'])

# From Telescope Data

In [1]:
import pandas as pd
import numpy as np

In [6]:
iter_df = pd.read_csv('scg-fun-75.tsv', delimiter='\t')

C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
iter_df.columns

Index(['Action.H2_201', 'Action.H2_221', 'Config.sigma',
       'Config.training_grade', 'CumulativeEpisodeReward', 'EpisodeEndTime',
       'EpisodeIndex', 'EpisodeStartTime', 'EpisodeType', 'IterationIndex',
       'IterationTimeStamp', 'SimulatorSessionId', 'State.BR_201',
       'State.BR_201_target_range', 'State.BR_221',
       'State.BR_221_target_range', 'State.C2', 'State.C2_target_range',
       'State.Density_221', 'State.MI_201', 'State.MI_201_target_range_high',
       'State.MI_201_target_range_low', 'State.MI_221',
       'State.MI_221_target_range_high', 'State.MI_221_target_range_low',
       'State.action_range_BR_201', 'State.action_range_BR_221',
       'State.action_range_BR_241', 'State.action_range_C2',
       'State.action_range_C4_221', 'State.action_range_C4_241',
       'State.action_range_H2_201', 'State.action_range_H2_221',
       'State.current_C4_221', 'State.current_H2_201', 'State.current_H2_221',
       'State.deltaMI_201', 'State.deltaMI_201_past', '

In [18]:
mapper = {}
for col in iter_df.columns:
    try:
        mapper.update({col: col.split('.')[1]})
    except:
        pass

iter_df = iter_df.rename(columns=mapper)

In [19]:
iter_df.columns

Index(['H2_201', 'H2_221', 'sigma', 'training_grade',
       'CumulativeEpisodeReward', 'EpisodeEndTime', 'EpisodeIndex',
       'EpisodeStartTime', 'EpisodeType', 'IterationIndex',
       'IterationTimeStamp', 'SimulatorSessionId', 'BR_201',
       'BR_201_target_range', 'BR_221', 'BR_221_target_range', 'C2',
       'C2_target_range', 'Density_221', 'MI_201', 'MI_201_target_range_high',
       'MI_201_target_range_low', 'MI_221', 'MI_221_target_range_high',
       'MI_221_target_range_low', 'action_range_BR_201', 'action_range_BR_221',
       'action_range_BR_241', 'action_range_C2', 'action_range_C4_221',
       'action_range_C4_241', 'action_range_H2_201', 'action_range_H2_221',
       'current_C4_221', 'current_H2_201', 'current_H2_221', 'deltaMI_201',
       'deltaMI_201_past', 'deltaMI_221', 'deltaMI_221_past', 'deltaPressure',
       'max_iteration', 'obs_range_BR_201', 'obs_range_BR_221', 'obs_range_C2',
       'obs_range_Density_221', 'obs_range_MI_201', 'obs_range_MI_221',
  

In [21]:
df_review = pd.DataFrame({})

cols = [
    'SimulatorSessionId',
    'EpisodeIndex',
    'IterationIndex',
    #'Terminal',
    'deltaMI_201',
    'deltaMI_221',
    'current_H2_201',
    'current_H2_221',
    'deltaPressure',
    'MI_201',
    'MI_221',
    'C2',
    'BR_201',
    'BR_221',
    #'Density_221',
    'MI_201_target_range_low',
    'MI_201_target_range_high',
    'MI_221_target_range_low',
    'MI_221_target_range_high',
    'step_count',
    'pressure_error',
    'sim_error',
    'sim_action_H2_201',
    'sim_action_H2_221',
    'sim_action_C2',
    'sim_action_C4_241',
    'sim_action_C4_221',
    'sim_action_BR_201',
    'sim_action_BR_241',
    'sim_action_BR_221',
    'H2_201',
    'H2_221',
    'sigma',
    'training_grade',
    'CumulativeEpisodeReward'
]

state_cols = [
    'deltaMI_201',
    'deltaMI_221',
    'current_H2_201',
    'current_H2_221',
    'deltaPressure',
    'MI_201',
    'MI_221',
    'C2',
    'BR_201',
    'BR_221',
    #'Density_221',
    'MI_201_target_range_low',
    'MI_201_target_range_high',
    'MI_221_target_range_low',
    'MI_221_target_range_high',
    'step_count',
    'pressure_error',
    'sim_error',
    'sim_action_H2_201',
    'sim_action_H2_221',
    'sim_action_C2',
    'sim_action_C4_241',
    'sim_action_C4_221',
    'sim_action_BR_201',
    'sim_action_BR_241',
    'sim_action_BR_221',
]

action_cols = [
    'H2_201',
    'H2_221',
]

config_cols = [
    'sigma',
    'training_grade'
]

for sess in iter_df['SimulatorSessionId'].unique():
    print("doing sess {}".format(sess))
    df_session = iter_df[iter_df['SimulatorSessionId'] == sess]
    df_session.sort_values(by=["EpisodeIndex", "IterationIndex"])
    for ep in range(1, int(max(df_session['EpisodeIndex']))+1):
        print("doing ep {}".format(ep))
        df_flat = df_session[df_session['EpisodeIndex'] == ep]
        if len(df_flat) > 1:
            if df_flat['IterationIndex'].iloc[1] == 1:
                print('Truncating ep because duplicate')
            elif df_flat['IterationIndex'].iloc[0] == 1:
                df_flat = df_flat[cols]
                df_review = pd.concat([df_review, df_flat])
                df_review['pressure_error'] = df_review['pressure_error'].map(int)
                df_review['sim_error'] = df_review['sim_error'].map(int)
            else:
                print('Truncating ep because did not start at IterationIndex=1')
        else:
            print('Empty episode')

doing sess 1318570140_10.244.78.151
doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing sess 1511151132_10.244.90.18
doing ep 1
doing ep 2
doing ep 3
doing sess 1561241105_10.244.93.14
doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 10
Truncating ep because duplicate
doing ep 11
doing ep 12
doing ep 13
doing ep 14
doing ep 15
doing ep 16
doing ep 17
doing ep 18
doing ep 19
doing ep 20
doing ep 21
doing ep 22
doing ep 23
doing ep 24
doing ep 25
doing ep 26
doing ep 27
doing ep 28
doing ep 29
doing ep 30
doing ep 31
doing ep 32
doing ep 33
doing ep 34
doing ep 35
doing ep 36
Truncating ep because duplicate
doing ep 37
doing ep 38
doing ep 39
doing ep 40
doing ep 41
doing ep 42
doing ep 43
doing ep 44
Truncating ep because duplicate
doing ep 45
doing ep 46
doing ep 47
doing ep 48
doing ep 49
doing ep 50
doing ep 51
doing ep 52
doing ep 53
doing ep 54
doing ep 55
doing ep 56
doing ep 57
Truncating ep because duplicate
doing ep 58
do

doing ep 12
doing ep 13
doing ep 14
doing ep 15
doing ep 16
doing ep 17
doing ep 18
doing ep 19
doing ep 20
doing sess 1272463957_10.244.75.216
doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 10
doing ep 11
doing ep 12
doing ep 13
doing ep 14
doing ep 15
doing ep 16
doing ep 17
doing ep 18
doing ep 19
doing ep 20
doing ep 21
doing ep 22
doing ep 23
doing ep 24
doing ep 25
doing ep 26
doing ep 27
doing ep 28
doing sess 1561224247_10.244.93.14
doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 10
Truncating ep because duplicate
doing ep 11
doing ep 12
doing ep 13
doing ep 14
doing ep 15
doing ep 16
doing ep 17
doing ep 18
doing ep 19
doing ep 20
doing ep 21
doing ep 22
doing ep 23
doing ep 24
doing ep 25
doing ep 26
doing ep 27
doing ep 28
doing ep 29
doing ep 30
doing ep 31
doing ep 32
doing ep 33
doing ep 34
doing ep 35
doing ep 36
Truncating ep because duplicate
doing e

doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 10
doing ep 11
doing ep 12
doing ep 13
doing sess 1483844858_10.244.88.113
doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 10
doing ep 11
doing ep 12
doing ep 13
doing ep 14
doing ep 15
doing ep 16
doing ep 17
doing ep 18
doing ep 19
doing ep 20
doing ep 21
doing ep 22
doing ep 23
doing ep 24
doing ep 25
doing ep 26
doing ep 27
doing ep 28
doing ep 29
doing ep 30
doing ep 31
doing ep 32
doing ep 33
doing ep 34
doing ep 35
doing ep 36
doing ep 37
doing ep 38
doing ep 39
doing ep 40
doing ep 41
doing ep 42
doing ep 43
doing ep 44
Truncating ep because duplicate
doing ep 45
doing ep 46
doing ep 47
doing ep 48
doing ep 49
doing ep 50
doing ep 51
doing ep 52
doing ep 53
doing ep 54
doing ep 55
doing ep 56
doing ep 57
doing ep 58
doing ep 59
doing ep 60
doing ep 61
doing ep 62
doing ep 63
doing ep 64
doing ep 65
doing ep 66
do

doing ep 124
doing ep 125
doing ep 126
doing ep 127
doing ep 128
doing ep 129
doing ep 130
doing sess 1511177815_10.244.90.18
doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 10
doing ep 11
doing ep 12
doing ep 13
doing ep 14
doing ep 15
doing ep 16
doing ep 17
doing ep 18
doing ep 19
doing ep 20
doing ep 21
doing ep 22
doing ep 23
doing ep 24
doing ep 25
doing ep 26
doing ep 27
doing ep 28
doing ep 29
doing ep 30
doing ep 31
doing ep 32
doing ep 33
doing ep 34
doing ep 35
doing ep 36
Truncating ep because duplicate
doing ep 37
doing ep 38
doing ep 39
doing ep 40
doing ep 41
doing ep 42
doing ep 43
doing ep 44
Truncating ep because duplicate
doing ep 45
doing ep 46
doing ep 47
doing ep 48
doing ep 49
doing ep 50
doing ep 51
doing ep 52
doing ep 53
doing ep 54
doing ep 55
doing ep 56
doing ep 57
Truncating ep because duplicate
doing ep 58
doing ep 59
doing ep 60
doing ep 61
doing ep 62
doing ep 63
doing ep 64
doing ep 65
doing e

doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 10
Truncating ep because duplicate
doing ep 11
doing ep 12
doing ep 13
doing ep 14
doing ep 15
doing ep 16
doing ep 17
doing ep 18
doing ep 19
doing ep 20
doing ep 21
doing ep 22
doing ep 23
doing ep 24
doing ep 25
doing ep 26
doing ep 27
doing ep 28
doing ep 29
doing ep 30
doing ep 31
doing ep 32
doing ep 33
doing ep 34
doing ep 35
doing ep 36
Truncating ep because duplicate
doing ep 37
doing ep 38
doing ep 39
doing ep 40
doing ep 41
doing ep 42
doing ep 43
doing ep 44
Truncating ep because duplicate
doing ep 45
doing ep 46
doing ep 47
doing ep 48
doing ep 49
doing ep 50
doing ep 51
doing ep 52
doing ep 53
doing ep 54
doing ep 55
doing ep 56
doing ep 57
Truncating ep because duplicate
doing ep 58
doing ep 59
doing ep 60
doing ep 61
doing ep 62
doing ep 63
doing ep 64
doing ep 65
doing ep 66
doing ep 67
doing ep 68
doing ep 69
doing ep 70
doing ep 71
doing ep 72
doing ep 73
doing ep 74
doing ep 75
doing ep 76
do

...

doing ep 59
doing ep 60
doing ep 61
doing ep 62
doing ep 63
doing ep 64
doing sess 1303497158_10.244.77.177
doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 10
doing ep 11
doing ep 12
doing ep 13
doing ep 14
doing ep 15
doing ep 16
doing ep 17
doing ep 18
doing ep 19
doing ep 20
doing ep 21
doing ep 22
doing ep 23
doing ep 24
doing ep 25
doing ep 26
doing ep 27
doing ep 28
doing ep 29
doing ep 30
doing ep 31
doing ep 32
doing ep 33
doing ep 34
doing ep 35
doing ep 36
doing ep 37
doing ep 38
doing ep 39
doing ep 40
doing ep 41
doing ep 42
doing ep 43
doing ep 44
Truncating ep because did not start at IterationIndex=1
doing ep 45
doing ep 46
doing ep 47
doing ep 48
doing ep 49
doing ep 50
doing ep 51
doing ep 52
doing ep 53
doing ep 54
doing ep 55
doing ep 56
doing ep 57
Truncating ep because did not start at IterationIndex=1
doing ep 58
doing ep 59
doing ep 60
doing ep 61
doing ep 62
doing ep 63
doing ep 64
doing ep 65
doing ep 

doing ep 62
doing ep 63
doing ep 64
doing ep 65
doing ep 66
doing ep 67
doing ep 68
doing ep 69
doing ep 70
doing ep 71
doing ep 72
doing ep 73
doing ep 74
doing ep 75
doing ep 76
doing ep 77
doing ep 78
doing ep 79
doing ep 80
doing ep 81
doing ep 82
doing ep 83
doing ep 84
doing ep 85
doing ep 86
doing ep 87
doing ep 88
doing ep 89
doing ep 90
doing ep 91
doing ep 92
doing sess 178445053_10.244.10.162
doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 10
doing ep 11
doing ep 12
doing ep 13
doing ep 14
doing ep 15
doing ep 16
doing ep 17
doing ep 18
doing ep 19
doing ep 20
doing ep 21
doing ep 22
doing ep 23
doing ep 24
doing ep 25
doing ep 26
doing ep 27
doing ep 28
doing ep 29
doing ep 30
doing ep 31
doing ep 32
doing ep 33
doing ep 34
doing ep 35
doing ep 36
Truncating ep because did not start at IterationIndex=1
doing ep 37
doing ep 38
doing ep 39
doing ep 40
doing ep 41
doing ep 42
doing ep 43
doing ep 44
Truncating ep beca

doing ep 2
doing ep 3
doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 10
doing ep 11
doing ep 12
doing ep 13
doing ep 14
doing ep 15
doing ep 16
doing ep 17
doing ep 18
doing ep 19
doing ep 20
doing ep 21
doing ep 22
doing ep 23
doing ep 24
doing ep 25
doing ep 26
doing ep 27
doing ep 28
doing ep 29
doing ep 30
doing ep 31
doing ep 32
doing ep 33
doing ep 34
doing ep 35
doing ep 36
doing ep 37
doing ep 38
doing ep 39
doing ep 40
doing ep 41
doing ep 42
doing ep 43
doing ep 44
doing ep 45
doing ep 46
doing ep 47
doing ep 48
doing ep 49
doing ep 50
doing ep 51
doing ep 52
doing ep 53
doing ep 54
doing ep 55
doing ep 56
doing ep 57
doing ep 58
doing ep 59
doing ep 60
doing ep 61
doing ep 62
doing ep 63
doing ep 64
doing ep 65
doing ep 66
doing ep 67
doing ep 68
doing ep 69
doing ep 70
doing ep 71
doing ep 72
doing ep 73
doing ep 74
doing ep 75
doing ep 76
doing ep 77
doing ep 78
doing ep 79
doing ep 80
doing ep 81
doing ep 82
doing ep 83
doing ep 84
doing ep 85


doing ep 42
doing ep 43
doing ep 44
doing ep 45
doing ep 46
doing ep 47
doing ep 48
doing ep 49
doing ep 50
doing ep 51
doing ep 52
doing ep 53
doing ep 54
doing ep 55
doing ep 56
doing ep 57
doing ep 58
doing ep 59
doing ep 60
doing ep 61
doing ep 62
doing ep 63
doing ep 64
doing ep 65
doing ep 66
doing ep 67
doing ep 68
doing ep 69
doing ep 70
doing ep 71
doing ep 72
doing ep 73
doing ep 74
doing ep 75
doing ep 76
doing ep 77
doing ep 78
doing ep 79
doing ep 80
doing ep 81
doing ep 82
doing ep 83
doing ep 84
doing ep 85
doing ep 86
doing ep 87
doing ep 88
doing ep 89
doing ep 90
doing sess 178440354_10.244.10.162
doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 10
Truncating ep because did not start at IterationIndex=1
doing ep 11
doing ep 12
doing ep 13
doing ep 14
doing ep 15
doing ep 16
doing ep 17
doing ep 18
doing ep 19
doing ep 20
doing ep 21
doing ep 22
doing ep 23
doing ep 24
doing ep 25
doing ep 26
doing ep 27
doing 

doing ep 56
doing ep 57
Truncating ep because did not start at IterationIndex=1
doing ep 58
doing ep 59
doing ep 60
doing ep 61
doing ep 62
doing ep 63
doing sess 87286769_10.244.5.51
doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 10
Truncating ep because did not start at IterationIndex=1
doing ep 11
doing ep 12
doing ep 13
doing ep 14
doing ep 15
doing ep 16
doing ep 17
doing ep 18
doing ep 19
doing ep 20
doing ep 21
doing ep 22
doing ep 23
doing ep 24
doing ep 25
doing ep 26
doing ep 27
doing ep 28
doing ep 29
doing ep 30
doing ep 31
doing ep 32
doing ep 33
doing ep 34
doing ep 35
doing ep 36
Truncating ep because did not start at IterationIndex=1
doing ep 37
doing ep 38
doing ep 39
doing ep 40
doing ep 41
doing ep 42
doing ep 43
doing ep 44
Truncating ep because did not start at IterationIndex=1
doing ep 45
doing ep 46
doing ep 47
doing ep 48
doing ep 49
doing ep 50
doing ep 51
doing ep 52
doing ep 53
doing ep 54
doing ep 

In [22]:
df_review.describe()

,EpisodeIndex,IterationIndex,deltaMI_201,deltaMI_221,current_H2_201,current_H2_221,deltaPressure,MI_201,MI_221,C2,...,sim_action_H2_221,sim_action_C2,sim_action_C4_241,sim_action_C4_221,sim_action_BR_201,sim_action_BR_241,sim_action_BR_221,sigma,training_grade,CumulativeEpisodeReward
count,464991.000000,464991.000000,464991.000000,464991.000000,464991.000000,464991.000000,464991.000000,464991.000000,464991.000000,464991.000000,...,464991.000000,464991.000000,464991.0,464991.00,464991.000000,464991.0,464991.000000,464991.0,464991.0,464991.000000
mean,38.648185,6.730438,1.684391,-0.716009,0.511754,0.166769,0.694593,392.645715,0.258291,54968.573585,...,0.833846,55075.159519,0.0,683.94,54.221126,0.0,45.778874,5.0,1.0,33.102779
std,37.194949,5.891560,1.248653,1.526548,0.298685,0.201331,0.255928,187.256278,0.508771,2628.239258,...,1.006656,1045.960975,0.0,0.00,1.917595,0.0,1.917595,0.0,0.0,20.072935
min,1.000000,1.000000,-0.979758,-1.564842,0.000000,0.000000,-0.146000,0.000000,0.000000,0.000000,...,0.000000,52500.000000,0.0,683.94,49.500000,0.0,45.000000,5.0,1.0,-18.415339
25%,10.000000,2.000000,0.273760,-1.293901,0.308105,0.035375,0.551857,180.911535,0.061957,55500.000000,...,0.176876,55500.000000,0.0,683.94,55.000000,0.0,45.000000,5.0,1.0,12.983992
50%,22.000000,4.000000,2.297841,-1.215356,0.513314,0.077531,0.619429,484.708412,0.090993,55500.000000,...,0.387654,55500.000000,0.0,683.94,55.000000,0.0,45.000000,5.0,1.0,33.377312
75%,63.000000,11.000000,2.829277,-0.515202,0.745687,0.250554,0.952000,564.313301,0.325311,55500.000000,...,1.252770,55500.000000,0.0,683.94,55.000000,0.0,45.000000,5.0,1.0,53.863701
max,175.000000,21.000000,3.441999,23.479086,1.000000,1.000000,1.166000,653.308374,8.315259,55500.000000,...,5.000000,55500.000000,0.0,683.94,55.000000,0.0,50.500000,5.0,1.0,56.268944


In [23]:
df_review.to_csv('filtered-scg-fun-75.csv')

In [24]:
ranges_to_eval = [-1]
df_dict = {}
for i in range(len(ranges_to_eval)):
    rng_to_slice = int(1.2 * ranges_to_eval[i])
    df_dict[str(ranges_to_eval[i])] = df_review[:rng_to_slice]

In [25]:
num_samples = -1
df_to_convert = df_dict[str(num_samples)]

In [36]:
import os
import numpy as np
import random

isdir = os.path.isdir("training")
if isdir:
    pass
else:
    os.makedirs("training")
    
isdir = os.path.isdir("validation")
if isdir:
    pass
else:
    os.makedirs("validation")

In [ ]:
for sess in df_to_convert['SimulatorSessionId'].unique():
    print("doing sess {}".format(sess))
    df_session = df_to_convert[df_to_convert['SimulatorSessionId'] == sess]
    df_session.sort_values(by=["EpisodeIndex", "IterationIndex"])
    for ep in df_session['EpisodeIndex'].unique():
        print("doing ep {}".format(ep))
        df_flat = df_session[df_session['EpisodeIndex'] == ep]
        df_flat = df_flat[cols]
        df_flat.replace(to_replace=['None'], value=np.nan, inplace=True)
        df_ep = pd.DataFrame({})
        df_ep['EpisodeIndex'] = '"' + sess + '_ep_' +df_flat['EpisodeIndex'].map(str) + '"'
        df_ep['IterationIndex'] = df_flat['IterationIndex'].astype(int)
        df_ep['is_initial_state'] = df_flat.eval("IterationIndex == 1")
        df_ep['is_terminal_state'] = df_flat.eval("step_count == {}".format(df_flat['step_count'].max()))
        df_ep['is_initial_state'] = df_ep['is_initial_state'].replace(True, 'true')
        df_ep['is_initial_state'] = df_ep['is_initial_state'].replace(False, 'false')
        df_ep['is_terminal_state'] = df_ep['is_terminal_state'].replace(True, 'true')
        df_ep['is_terminal_state'] = df_ep['is_terminal_state'].replace(False, 'false')
        df_ep['state'] = df_flat[state_cols].to_numpy().tolist()
        df_ep['action'] = df_flat[action_cols].astype('float').to_numpy().tolist()
        df_ep['action'].iloc[0] = []
        df_ep['config'] = df_flat[config_cols].to_numpy().tolist()
        if np.random.uniform(0, 1) <= 0.2:
            df_ep.to_csv('./validation/{}_ep_{}.csv'.format(sess, int(ep)), index=False, header=False)
        else:
            df_ep.to_csv('./training/{}_ep_{}.csv'.format(sess, int(ep)), index=False, header=False)

doing sess 1318570140_10.244.78.151
doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing sess 1511151132_10.244.90.18
doing ep 1
doing ep 2


C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

doing ep 3
doing sess 1561241105_10.244.93.14
doing ep 1
doing ep 2


C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

doing ep 3
doing ep 4
doing ep 5
doing ep 6
doing ep 7
doing ep 8
doing ep 9
doing ep 11


...

doing ep 111
doing ep 112
doing ep 113
doing ep 114
doing ep 115
doing ep 116
doing ep 117
doing ep 118
doing ep 119
doing ep 120


C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

doing ep 121
doing ep 122
doing ep 123
doing ep 124
doing ep 125
doing ep 126
doing ep 127
doing ep 128


C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\JOMCDOWE\Miniconda3\envs\bonsai-preview\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

doing ep 129
doing ep 130
doing sess 1511177815_10.244.90.18
doing ep 1
doing ep 2
doing ep 3
doing ep 4
doing ep 5
doing ep 6
